In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
date = '20190715'
ww_fresh = xr.open_dataset('/ocean/cdonaldson/scrubbers/daily_obs/scrubber_obs_freshcalc_{}.nc'.format(date))
ww_fresh

<xarray.Dataset>
Dimensions:    (depth: 1, y: 898, x: 398)
Coordinates:
  * depth      (depth) float64 6.0
    time       datetime64[ns] ...
  * y          (y) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * x          (x) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Data variables:
    washwater  (depth, y, x) float32 ...

In [4]:
phys = xr.open_dataset('/ocean/cdonaldson/scrubbers/day_avg_extractions/SalishSeaCast_day_avg_phys_tracers__20190101_20191231.nc')

In [5]:
phys

<xarray.Dataset>
Dimensions:   (time: 365, depth: 1, gridY: 898, gridX: 398)
Coordinates:
  * time      (time) datetime64[ns] 2019-01-01T12:00:00 ... 2019-12-31T12:00:00
  * depth     (depth) float32 0.5
  * gridY     (gridY) int64 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX     (gridX) int64 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
Data variables:
    votemper  (time, depth, gridY, gridX) float32 ...
    vosaline  (time, depth, gridY, gridX) float32 ...
Attributes:
    name:         SalishSeaCast_day_avg_phys_tracers__20190101_20191231
    description:  Day-averaged river dye extracted from SalishSeaCast v202111...
    history:      2026-01-28 13:50 -08:00: Generated by `reshapr extract /oce...
    Conventions:  CF-1.6

# Goal
Transform ww_fresh into ww_est by adding salt and temperature, and replacing the stand-in depth with a static estimated depth for the model.

### Salt
Assume that the water at the outflow is the same salinity as the water at the intake, and that there is little delay in processing time within the scrubber system. Since we are doing daily runs, we can use the day-averaged salinity over the domain instead of hourly salinity. This means that these results will most likely represent open-loop scrubber systens.

***Day-avg sal at (X, Y) grid point***

### Temperature
A little bit trickier, since there isn't much literature about the temperature at the outflow and the guidelines (60C?) seem to be much more generous than any real values. Discussions of temperature tended to be buried within discussions of pH. There are two case studies summarized in the EPA report from 2011, "Exhaust Gas Scrubber Washwater Effluent" (inst. both references) and a modelling study from Ristea et al., "Computational Fluid Dynamics Simulation Approach for Scrubber Wash Water pH Modelling." Based on these, I am going to make the temperature at the outfall a function of the temperature of the intake water, with just an addition of X degrees.

Riseta et al.: CFD study, they set seawater temp at 25°C and discharge water temp to 35°C.
Flagiello et al.: experimental study of alkaline additives to washwater, not in-situ. Water at the intake was 20°C, and exited in the range of 30°C to 50°C.
EPA Study: cites two experimental studies, text found availible for one of them. THE MS ZAANDAMSEAWATER, Holland America Cruise Liner: increase in washwater temperature was less than 3.5°C (HA & H-K, 2010). Text not found for me to read and validate their numbers. THE PRIDE OF KENT, Ferry between Dover and Calais: MArch +1.9°C, July 4.1°C, September 11.9°C, November 16.9°C (Hufnagl et al., 2005). Avg = 8.7°C.

Take the average of both studies = 6.1°C. I'm not very confident in this, so I'm going to go with order of magnitude and say +10°C for all seasons at all locations.

***Day-avg temp at (X, Y) grid point + 10°C***

### Depth
This one is also tricky, since we already know that we will not be able to exactly represent the depth. Ships will sit higher or lower in the water based on their cargo load, and we don't have that information for our data. One solution could be to pick one average depth, but I think it may be better to pick a range of depth to distribute the effluent outfall over since we have fine, 1m resolution in the model. Now, the question becomes one of "what range" and I hope would be a better representation for us to do time-averaged studies on. Does this then impact the assumption I am about to make where I use the surface salinity and temperature as my starting intake values?

The main types of ship to focus on are merchant bulk, merchant container, and cruise ships based on the ECCC emissions simulation tool.

#### Draft Values
Merchant Bulk: 10 - 20 meters (https://shippingandcommodityacademy.com/blog/list-of-bulk-vessel-sizes-an-overview-of-maritime-freight-options/, https://ouco-industry.com/bulk-carriers-explained-types-uses-why-they-matter/#:~:text=Table_title:%20Class%20of%20Bulk%20Carrier%20Table_content:%20header:,120%2C000%E2%80%93200%2C000%20DWT%20%7C%20Draft:%20~16%20m%20%7C)

Merchant Container: 8 - 16 meters (https://transportgeography.org/contents/chapter5/maritime-transportation/draft-containership-capacity/)

Cruise: 4 - 8m (https://www.cruisemapper.com/wiki/753-cruise-ship-sizes-comparison-dimensions-length-weight-draft, put lower range at half of max draft to align with the bulk and container references, looked up examples of common cruise ships that have transited the Salish Sea: Holland America: 8m, 8m | Princess: 8.55m, 8m, | Royal Caribbean: 8.8m, 8.82m, | Celebrity: 8.62m, 8.30m | Norwegian: 3m, 8.72m)

We should also consider that the ship parameters are likely limited by the ports they are able to stop through. Since our dataset does not include Puget Sound, we will assume that the shipping traffic in the SoG that we see is bound for the Port of Vancouver, as opposed to the Port of Seattle or the Port of Tacoma. The maximum draft listed for the port of Vancouver is ~ 15 meters (https://www.findaport.com/port-of-vancouver-canada#limits). Port of Seattle is 15 m (https://en.wikipedia.org/wiki/United_States_container_ports) and Port of Tacoma is 15 m (https://en.wikipedia.org/wiki/United_States_container_ports). I will also assume that before the port, the ships are not ballasted lower.

This gives new ranges of 10 - 15m (merchant bulk), 8 - 15m (merchant container), and 4 - 8m (cruise).

#### Scrubber Outlet Depth
For these types of ships, we assume that the scrubber outlet depth will always be below sea level, but is not often on the bottom of the hull (interference with prop system). Let's also assume that sailing in the fully unloaded state (i.e. unloaded repositioning) is unlikely, as operation is costly within global shipping. 

<!-- I will incorporate this mathematically by assuming that ships will always be at least 130% of their minimum draft. We can determine upper and lower estimates for the hull position of the scrubber outlets as  -->

<!-- $$u = 1.3 \times x_{min} + a$$
$$l = x_{max} - a$$ -->

<!-- $$u = 0.5 \times x_{max} + a - (0.25 \times x_{max})$$
$$l = x_{max} - a - (0.25 \times x_{max})$$ -->
<!-- 
where $a$, allowance, is the distance the scrubber sits from the keel and the minimum waterline, x_{min} is the minimum draft, and $x_{max}$ is the maximum draft. 

We should cap the "maximum draft" to 15 to arrive at the following ranges: 15 - 13m (bulk, 2m allowance), 12.4 - 13m (container, 2m allowance), and 6.2 - 7m (cruise, 1m allowance).  -->

Mathematical estimates commented out, showing deep estimates of 12 - 13m (bulk, 2m allowance), 8 - 13m (container, 2m allowance), and 5 - 7m (cruise, 1m allowance).

<!-- 12 - 13m (bulk, 2m allowance), 8 - 13m (container, 2m allowance), and 5 - 7m (cruise, 1m allowance).  -->

<!-- Our in-situ scrubber depth ranges then become 2 - 13m, 2 - 13m, and 1 - 7m to capture the fully unloaded state (scrubber depth at allowance) as well as the loaded to Salish Sea port capacity state (scrubber depth at max draft - allowance).  This gives new ranges -->

<!-- and we have assumed that the vessel rides on average in the middle of its draft range.  -->
<!-- This gives us 7 - 13 (2m safety), 6 - 10 (2m safety), and 3 - 5 (1m safety) which we can then simplify to a total depth range of 3m - 13m to use in the model. If we want to pick a single value from this, we land on 8m depth. -->

Alternatively, we can take 2.8m from Val et al., the maximum draft of 15 in the ports, and an allowance estimate based on the diagram in the patent to estimate the 3 - 13m for the large ships coming through the Salish Sea. Can either use this range, or we can say 8m depth which is closest to either 7.5m or 8.5m. I am inclined to go with 8.5m, since most of the traffic we are estimating comes from the sum of bulk and container shipping.

The range is practically all in the average estuarine layer, but since we are doing day averaged things near the FRP, I am inclined to not want to condense it to one. Plus, shouldn't it have mixed as the ship moves through? I'm struggling with this one. Make the files for both.

***3m - 13m range*** OR ***8.5m***

# Salt